# Домашнее задание №4. «Рекомендации на основе скрытых факторов»

## “Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

In [2]:
data = pd.read_table('user_taggedbookmarks-timestamps.dat', sep='\\n', delimiter='\\t')

C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
C:\Users\Arsenii\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
del data['timestamp']

In [4]:
data.head()

,userID,bookmarkID,tagID
0,8,1,1
1,8,2,1
2,8,7,1
3,8,7,6
4,8,7,7


In [5]:
data = data.groupby(['userID', 'bookmarkID'], as_index=False)['tagID'].count()
data = data.rename(columns={'tagID': 'tags_num'})

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104799 entries, 0 to 104798
Data columns (total 3 columns):
userID        104799 non-null int64
bookmarkID    104799 non-null int64
tags_num      104799 non-null int64
dtypes: int64(3)
memory usage: 3.2 MB


In [7]:
data.head()

,userID,bookmarkID,tags_num
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2


## Отсутствующих значений нет.

In [8]:
user_id_lookup = dict(zip(range(data['userID'].nunique()), data['userID'].unique()))
item_id_lookup = dict(zip(range(data['bookmarkID'].nunique()), data['bookmarkID'].unique()))
data['userID'] = data['userID'].map({v: k for k, v in user_id_lookup.items()})
data['bookmarkID'] = data['bookmarkID'].map({v: k for k, v in item_id_lookup.items()})

In [9]:
ids = list(np.sort(data['userID'].unique()))
urls = list(np.sort(data['bookmarkID'].unique()))

In [10]:
rows = data['userID']
cols = data['bookmarkID']
tags_num = data['tags_num']

In [11]:
len(tags_num), len(cols), len(rows)

(104799, 104799, 104799)

In [12]:
len(urls), len(ids)

(69223, 1867)

In [13]:
data_sparse = sparse.csr_matrix((tags_num, (cols, rows)), shape=(len(urls), len(ids)))

In [14]:
model = AlternatingLeastSquares(factors=100)
model.fit(data_sparse)

100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:01<00:00,  9.31it/s]


In [15]:
userid = 8
user_items = data_sparse.transpose()
recommendations = model.recommend(userid, user_items)

In [17]:
recommendations = [(item_id_lookup[rec[0]], rec[1]) for rec in recommendations]

In [18]:
recommendations

[(742, 0.40164775),
 (731, 0.40076268),
 (741, 0.3738892),
 (709, 0.36471456),
 (662, 0.34974694),
 (661, 0.34922293),
 (706, 0.3463288),
 (693, 0.32178512),
 (759, 0.3138817),
 (680, 0.30963033)]